### 3D UNet test
Trialing 3D UNet implementation found here: https://github.com/wolny/pytorch-3dunet

###Code to install and activate the correct conda environment 

In cmd line with a conda environment:
```
conda install -c conda-forge mamba
mamba create -n pytorch3dunet -c pytorch -c nvidia -c conda-forge -c awolny pytorch-3dunet
conda activate pytorch3dunet
```

#### Running the model
__config files determine filepaths etc.__

train3dunet --config <CONFIG>



predict3dunet --config <CONFIG>


In [5]:
#create testing HDF5
%cd C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest
%ls

C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest
 Volume in drive C is OS
 Volume Serial Number is 9427-9109

 Directory of C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest

06/06/2023  01:27 PM    <DIR>          .
06/06/2023  01:27 PM    <DIR>          ..
06/08/2023  11:45 AM    <DIR>          Checkpoint
06/06/2023  01:27 PM    <DIR>          output
06/14/2023  01:24 PM                 0 test.h5
06/06/2023  02:39 PM        62,825,224 train.h5
06/06/2023  02:39 PM        62,825,224 val.h5
               3 File(s)    125,650,448 bytes
               4 Dir(s)  247,493,861,376 bytes free


In [6]:
%pip install tables


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
import tables
import numpy as np
img_dtype = {}
img_dtype["raw"] = "float64"  
img_dtype["label"] = "int16"

imgtypes = ["raw","label"]

#brute force training and validation things
training = [r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133"]
# ,r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077"

validating = [r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133"]



In [13]:
import SimpleITK as sitk
import h5py as h5


reader = sitk.ImageFileReader()
reader.SetFileName(r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_label_resampled.mha")
reader.LoadPrivateTagsOn()
reader.ReadImageInformation()

size = reader.GetSize()
str_size = (0,size[0],size[1])
max_size = (None,size[0],size[1])
print(str_size)

raw_file = h5.File(r"C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest\train.h5", "w")
block_shape={} 

def filterA(a,val):
    arr = np.array(a)
    arr[arr != val] = 0
    arr = arr
    return arr.tolist()





dset = {}
for imgtype in imgtypes: #init arrays for each storage
        dset[imgtype]= raw_file.create_dataset(name= imgtype, shape=str_size,maxshape=max_size,dtype=img_dtype[imgtype])


for file in training:
    for type in imgtypes:
        path = file + r"\UH_"+ file.rsplit("\\",1)[-1] + "_pre_ax_raw" + ("_label" if type == "label" else "") + "_resampled.mha"
        if path == r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_raw_label_resampled.mha":
            path = r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_label_resampled.mha"
        
        image = sitk.ReadImage(path, imageIO="MetaImageIO")
        n = [sitk.GetArrayFromImage(image[:,:,i]) for i in range(image.GetDepth())]
        arr = np.array(n)
        print(arr.shape)
        #n = np.expand_dims(n,0)

        if (type == "label"):
            n = filterA(n,1)
        s = dset[type].shape[0]
        dset[type].resize(dset[type].shape[0]+arr.shape[0], axis=0)

        temp = dset[type]
        for i in range(arr.shape[0]):
            slice = n[i]
            temp[s+i] = slice
        dset[type] = temp
        print(path)
        print('done')

    




raw_file.close()







print("Doing validation now")

val_file = h5.File(r"C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest\val.h5", "w")

dset = {}
for imgtype in imgtypes: #init arrays for each storage
        dset[imgtype]= val_file.create_dataset(name= imgtype, shape=str_size,maxshape=max_size,dtype=img_dtype[imgtype])

for file in validating:
    for type in imgtypes:
        path = file + r"\UH_"+ file.rsplit("\\",1)[-1] + "_pre_ax_raw" + ("_label" if type == "label" else "") + "_resampled.mha"
        if path == r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_raw_label_resampled.mha":
            path = r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_label_resampled.mha"
        
        image = sitk.ReadImage(path, imageIO="MetaImageIO")
        n = [sitk.GetArrayFromImage(image[:,:,i]) for i in range(image.GetDepth())]
        arr = np.array(n)
        print(arr.shape)
        #n = np.expand_dims(n,0)

        if (type == "label"):
            n = filterA(n,1)
        s = dset[type].shape[0]
        dset[type].resize(dset[type].shape[0]+arr.shape[0], axis=0)

        temp = dset[type]
        for i in range(arr.shape[0]):
            slice = n[i]
            temp[s+i] = slice
        dset[type] = temp
        print(path)
        print('done')


    

val_file.close()




(0, 180, 180)
(192, 180, 180)
G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_resampled.mha
done
(192, 180, 180)
G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_label_resampled.mha
done
Doing validation now
(192, 180, 180)
G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_resampled.mha
done
(192, 180, 180)
G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_label_resampled.mha
done


In [12]:
raw_file.close()
#val_file.close()

In [9]:
# reading the image types

file = r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_resampled.mha"
image = sitk.ReadImage(file, imageIO="MetaImageIO")
print(image.GetPixelIDTypeAsString()) 

file = r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_label_resampled.mha"
image = sitk.ReadImage(file, imageIO="MetaImageIO")
print(image.GetPixelIDTypeAsString()) 

64-bit float
16-bit unsigned integer


### Training the model

Most important is checking that CUDA, cuDNN, and other necessary things are working correctly

Also, if pytorch is detecting multiple tensor devices, especially a CUDA enabled device and a CPU, run 

```
conda env config vars set CUDA_VISIBLE_DEVICES = 'GPU_UUID_HERE'
```
This will correctly introduce the devices to use on Win10


In [4]:
#correct install of torch and such

%pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

Looking in indexes: https://download.pytorch.org/whl/nightly/cu121
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [7]:
##from StackOverflow, returns information on the GPU, checks that cuDNN is installed

import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_properties(0))
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

True
1
_CudaDeviceProperties(name='NVIDIA GeForce RTX 3060 Laptop GPU', major=8, minor=6, total_memory=6143MB, multi_processor_count=30)
__Python VERSION: 3.10.1 (tags/v3.10.1:2cd268a, Dec  6 2021, 19:10:37) [MSC v.1929 64 bit (AMD64)]
__pyTorch VERSION: 2.1.0.dev20230602+cu121
__CUDA VERSION
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:36:15_Pacific_Daylight_Time_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0
__CUDNN VERSION: 8801
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0
Available devices  1
Current cuda device  0


In [8]:
%pip install gputil

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [30]:
###get uuid, needed for limiting the cuda visible devices

import GPUtil
GPUtil.getGPUs()[0].uuid

'GPU-169e5b45-a28f-0939-3716-0aa42920e279'

### Convert output into mask


In [2]:
%pip install h5py
import h5py as h5

You should consider upgrading via the 'c:\Users\theev\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


In [15]:
test_file.close()

In [16]:
test_file = h5.File(r"C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest\test.h5", "w")
dset = test_file.create_dataset(name = "raw", shape = (0, 180, 180), maxshape = (None,180,180), dtype="float64")



for file in validating:
    for type in imgtypes:
        if(type == "label"):
            continue
        path = file + r"\UH_"+ file.rsplit("\\",1)[-1] + "_pre_ax_raw" + ("_label" if type == "label" else "") + "_resampled.mha"
        if path == r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_raw_label_resampled.mha":
            path = r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_077\UH_RectalCA_077_pre_ax_label_resampled.mha"
        
        image = sitk.ReadImage(path, imageIO="MetaImageIO")
        n = [sitk.GetArrayFromImage(image[:,:,i]) for i in range(image.GetDepth())]
        arr = np.array(n)
        print(arr.shape)
        #n = np.expand_dims(n,0)
        s = dset.shape[0]
        dset.resize(dset.shape[0]+arr.shape[0], axis=0)
        for i in range(arr.shape[0]):
            dset[s+i] = n[i]
        print(path)
        print('done')


test_file.close()

(192, 180, 180)
G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_resampled.mha
done


In [24]:
import SimpleITK as sitk
import h5py as h5

path = r"C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest\output\test_predictions.h5"
table = h5.File(path,'r')
list(table.keys())

predictions = table['predictions']
print(predictions)
print(predictions[0].shape)


p = predictions[0]

def filterB(a,val):
    arr = np.array(a)
    arr[arr < val] = 0

    arr[arr != 0] = 1
    return arr



p = filterB(p,.85)

mask = sitk.GetImageFromArray(p)
table.close()



<HDF5 dataset "predictions": shape (1, 192, 180, 180), type "<f4">
(192, 180, 180)


In [26]:
%cd  C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest\output

C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest\output


In [27]:
sitk.WriteImage(mask,"mask.mha")

In [25]:
import SimpleITK as sitk
grid_image = mask #sitk image object

image_viewer = sitk.ImageViewer()
image_viewer.SetTitle('Python image viewer') #arbitrary
image_viewer.SetApplication(r'C:\Program Files\ITK-SNAP 3.8\bin\ITK-SNAP') #can open directly in application

image_viewer.Execute(grid_image)


In [5]:
import SimpleITK as sitk
import numpy as np

r = sitk.ImageFileReader()
r.SetFileName(r"C:\Users\theev\OneDrive\Documents\LAB\Code\3DUNetTest\output\mask.mha")
temp = r.Execute()

mask = sitk.GetArrayFromImage(temp)

r.SetFileName(r"G:\Shared drives\INVent\mfk58\Dataset\UH_Pre_Resampled\RectalCA_133\UH_RectalCA_133_pre_ax_raw_label_resampled.mha")
temp = r.Execute()

def filterC(a,val):
    arr = np.array(a)
    arr[arr != val] = 0
    return arr


output = sitk.GetArrayFromImage(temp)
output = filterC(output, 1)


TP = (mask[output>0]>0).sum()/(output>0).sum()
FP = (mask[output==0]>0).sum()/(output==0).sum()
TN = (mask[output==0]==0).sum()/(output==0).sum()
FN = (mask[output>0]==0).sum()/(output>0).sum()
DS = 2*TP / (2*TP + FP + FN)
Precision = TP/(TP+FP)
Recall = TP/(TP+FN)
SR = TN/(TN+FP) #specificity rate
JI = TP/(TP+FN+FP) #Jaccard's index
print(TP)
print(FP)
print(TN)
print(FN)
print(DS)
print(Precision)
print(Recall)
print(SR)
print(JI)

print()
print()

print((mask[output>0]>0).sum())
print((mask[output==0]>0).sum())
print((mask[output==0]==0).sum())
print((mask[output>0]==0).sum())

0.5866493459136697
0.009011675115978297
0.9909883248840217
0.41335065408633026
0.7353057299539931
0.9848711350955935
0.5866493459136697
0.9909883248840217
0.5814098690644375


51482
55269
6077775
36274
